# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.  
 
Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here

import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import stopwords
import re
import nltk
from nltk.probability import ConditionalFreqDist
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [11]:
# Define the desired column names
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

# Try different encodings
encodings = ['utf-8', 'latin1', 'windows-1252']
for encoding in encodings:
    try:
        # Read the CSV file with specified column names and sample 20000 rows
        tweets = pd.read_csv("C:/Users/diana/Documents/Repos/GitHub/lab-nlp/training.1600000.processed.noemoticon.csv", encoding=encoding, names=column_names, header=None).sample(10000)
        break  # If successful, exit the loop
    except UnicodeDecodeError:
        print(f"Failed to read with encoding {encoding}. Trying next encoding...")

print("Data loaded successfully.")

Failed to read with encoding utf-8. Trying next encoding...
Data loaded successfully.


In [20]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.
    
    Returns:
        A string that has been cleaned up.
    """
    string = re.sub(r'http\S+', '', s)
    
    return re.sub('[^A-Za-z]+', ' ', string).lower().strip()

from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return word_tokenize(s)

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet') 
nltk.download('omw-1.4')

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer() 
    sentence_stemmed_lemmatized =[]
    
    for w in l:
        stemmed = ps.stem(w)
        stemmed_lemmatized = lemmatizer.lemmatize(stemmed)
        sentence_stemmed_lemmatized += [stemmed_lemmatized]
    
    return sentence_stemmed_lemmatized

nltk.download('stopwords')

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    
    
    return ([w for w in l if w not in stopwords.words('english')])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\diana\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\diana\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [29]:
# your code here

# Apply the clean_up function to the 'text' column
tweets['text_processed'] = tweets['text'].apply(clean_up)

# Apply the tokenize function to the 'text_processed' column
tweets['text_processed'] = tweets['text_processed'].apply(tokenize)

# Apply the stem_and_lemmatize function to the 'text_processed' column
tweets['text_processed'] = tweets['text_processed'].apply(stem_and_lemmatize)

# Apply the remove_stopwords function to the 'text_processed' column
tweets['text_processed'] = tweets['text_processed'].apply(remove_stopwords)

# Display the DataFrame with the new text_processed column
tweets.head()

,target,id,date,flag,user,text,text_processed
1264279,4,1999215921,Mon Jun 01 19:50:21 PDT 2009,NO_QUERY,TheTAZZone,@verwon that's why you're so much fun to play ...,"[verwon, whi, much, fun, play]"
1263553,4,1999062308,Mon Jun 01 19:34:39 PDT 2009,NO_QUERY,rifter,"@tobinrogers All I did, was add the IFC part, ...","[tobinrog, wa, add, ifc, part, coupl, link]"
1182850,4,1982328653,Sun May 31 10:59:32 PDT 2009,NO_QUERY,painyboi,@weezereire brings a tear to my eye LOL thanks!,"[weezereir, bring, tear, eye, lol, thank]"
898182,4,1693535812,Sun May 03 22:52:21 PDT 2009,NO_QUERY,blogworks,@Su Sure you can. If you author more than one ...,"[su, sure, author, one, blog, may, submit, man..."
616861,0,2226823479,Thu Jun 18 12:25:56 PDT 2009,NO_QUERY,iHeather_XD,Not going to LK's. Maybe going to Catherine's...,"[go, lk, mayb, go, catherin, answer, dang, phone]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [30]:
# your code here
from nltk.probability import FreqDist

# Combine all words in the text_processed column into a single list
all_words = [word for sublist in tweets['text_processed'] for word in sublist]

# Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

# Select the top 5,000 words
top_words = freq_dist.most_common(5000)

# Display the top 5 words
top_words[:5]

[('go', 836), ('day', 725), ('get', 661), ('wa', 613), ('good', 597)]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [37]:
# your code here
def document_features(document, top_words):
    document_words = set(document)
    features = {}
    for word in top_words:
        features[word] = (word in document_words)
    return features

# Create the feature set for each tweet
feature_set = [(document_features(tweet, top_words), sentiment) for tweet, sentiment in zip(tweets['text_processed'], tweets['target'])]

# Display an example feature set
print(feature_set[0])

({('go', 836): False, ('day', 725): False, ('get', 661): False, ('wa', 613): False, ('good', 597): False, ('thi', 568): False, ('like', 540): False, ('work', 534): False, ('love', 504): False, ('quot', 501): False, ('today', 469): False, ('got', 469): False, ('u', 455): False, ('time', 407): False, ('thank', 384): False, ('lol', 382): False, ('back', 362): False, ('one', 358): False, ('want', 355): False, ('feel', 340): False, ('im', 335): False, ('know', 333): False, ('think', 322): False, ('miss', 321): False, ('realli', 320): False, ('amp', 310): False, ('night', 286): False, ('still', 284): False, ('come', 279): False, ('watch', 276): False, ('hope', 276): False, ('new', 275): False, ('na', 272): False, ('see', 269): False, ('well', 263): False, ('look', 258): False, ('need', 258): False, ('make', 257): False, ('home', 255): False, ('morn', 253): False, ('oh', 240): False, ('last', 238): False, ('ha', 234): False, ('twitter', 232): False, ('wait', 229): False, ('wish', 227): False,

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [43]:
# your code here

import random
from nltk import NaiveBayesClassifier
from nltk.classify import accuracy

# Shuffle the feature set
random.shuffle(feature_set)

# Split the feature set into a training set and a test set (80% training, 20% test)
train_size = int(len(feature_set) * 0.8)
train_set, test_set = feature_set[:train_size], feature_set[train_size:]

# Train the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(train_set)

# Test the classifier accuracy on the test set
print("Classifier accuracy:", accuracy(classifier, test_set))

# Show the most informative features
classifier.show_most_informative_features()

Classifier accuracy: 0.507
Most Informative Features
              ('aaa', 2) = False               0 : 4      =      1.0 : 1.0
            ('aaaah', 2) = False               0 : 4      =      1.0 : 1.0
           ('aaaand', 2) = False               0 : 4      =      1.0 : 1.0
            ('aaron', 4) = False               0 : 4      =      1.0 : 1.0
          ('abandon', 3) = False               0 : 4      =      1.0 : 1.0
            ('abbey', 2) = False               0 : 4      =      1.0 : 1.0
             ('abbi', 2) = False               0 : 4      =      1.0 : 1.0
       ('abbyluvsaj', 1) = False               0 : 4      =      1.0 : 1.0
              ('abc', 2) = False               0 : 4      =      1.0 : 1.0
             ('abil', 2) = False               0 : 4      =      1.0 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [44]:
# your code here

from nltk.classify import accuracy

# Evaluate the classifier accuracy on the test set
accuracy_score = accuracy(classifier, test_set)
print("Classifier accuracy:", accuracy_score)

# Check if the classifier performance is OK or great
if accuracy_score > 0.6:
    print("The Naive Bayes model is considered OK.")
    if accuracy_score > 0.7:
        print("You've done a great job!")
else:
    print("The Naive Bayes model performance needs improvement.")


Classifier accuracy: 0.507
The Naive Bayes model performance needs improvement.


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [45]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [46]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [47]:
# your code here